# CITS Data Analysis - Clean Version

This notebook provides a clean implementation for analyzing CITS (Current Imaging Tunneling Spectroscopy) data.

## Features:
1. Load and parse SPM data files (.txt and .dat)
2. Handle data orientation correctly based on scan direction
3. Interactive bias voltage selection
4. Performance analysis of different flipping methods
5. STS line analysis along selected paths
6. All visualizations using Plotly for consistency

## Key Innovation:
- Uses `prepare_cits_for_display()` function to ensure correct data orientation
- Preserves original data in dat_parser while handling display orientation at analysis layer
- Efficient 3D numpy slicing for Y-axis flipping

In [2]:
# Import all necessary libraries
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import time
from typing import Dict, List, Tuple, Optional

# Plotly for all visualizations
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# IPython widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add parent directory to path for imports
sys.path.append(str(Path.cwd().parent))

# Import custom parsers
from core.parsers.txt_parser import TxtParser
from core.parsers.dat_parser import DatParser

print("All libraries imported successfully!")

All libraries imported successfully!


In [3]:
# Define the core function for handling data orientation
def prepare_cits_for_display(data_3d: np.ndarray, scan_direction: str) -> np.ndarray:
    """
    Prepare CITS data for display with correct orientation.
    Ensures origin (0,0) is at bottom-left corner for consistent visualization.
    
    Args:
        data_3d: Original CITS data array of shape (n_bias, y, x)
        scan_direction: 'upward' or 'downward' scan direction
    
    Returns:
        np.ndarray: View of data with correct orientation for display
        
    Note:
        - For downward scans: Flip Y-axis to correct orientation
        - For upward scans: Keep original orientation
        - This operation creates a view, not a copy (memory efficient)
    """
    if data_3d.ndim != 3:
        raise ValueError("Input data must be a 3D array of shape (n_bias, y, x)")
    
    if scan_direction not in ['downward', 'upward']:
        raise ValueError("Scan direction must be 'downward' or 'upward'")
    
    if scan_direction == 'downward':
        # Flip Y-axis for downward scans to ensure origin at bottom-left
        return data_3d[:, ::-1, :]
    else:
        # Keep as-is for upward scans
        return data_3d

def is_cits_data(data: Dict) -> bool:
    """
    Check if the loaded data is CITS format (3D data with bias voltages)
    """
    if 'measurement_mode' in data and data['measurement_mode'] == 'CITS':
        return True
    
    if 'data_3d' in data:
        data_array = np.array(data['data_3d'])
        if data_array.ndim == 3 and data_array.shape[0] >= 2:
            return True
    
    return False

print("Core functions defined successfully!")

Core functions defined successfully!


## Step 1: Load and Parse Data Files

In [4]:
# Define path to test file
test_file_path = Path("../../testfile/20250521_Janus Stacking SiO2_13K_113.txt")

# Load .txt file
print("Loading TXT file...")
txt_parser = TxtParser(str(test_file_path))
txt_data = txt_parser.parse()

# Extract experiment info
exp_info = txt_data.get('experiment_info', {})
dat_files_info = txt_data.get('dat_files', [])

print(f"✓ File loaded: {test_file_path.name}")
print(f"✓ Found {len(dat_files_info)} DAT files")
print(f"✓ Scan range: {exp_info.get('XScanRange', 'N/A')} x {exp_info.get('YScanRange', 'N/A')} {exp_info.get('XPhysUnit', '')}")
print(f"✓ Pixels: {exp_info.get('xPixel', 'N/A')} x {exp_info.get('yPixel', 'N/A')}")
print(f"✓ Angle: {exp_info.get('Angle', 'N/A')}°")

# Display available DAT files
print("\nAvailable DAT files:")
for i, dat_file in enumerate(dat_files_info):
    print(f"  [{i}] {dat_file['filename']} - {dat_file.get('caption', 'N/A')}")
    print(f"      Mode: {dat_file.get('measurement_mode', 'N/A')}, Type: {dat_file.get('measurement_type', 'N/A')}")
    if dat_file.get('grid_size'):
        print(f"      Grid: {dat_file['grid_size'][0]}×{dat_file['grid_size'][1]}")

Loading TXT file...
✓ File loaded: 20250521_Janus Stacking SiO2_13K_113.txt
✓ Found 4 DAT files
✓ Scan range: 10.000 x 10.000 nm
✓ Pixels: 500 x 500
✓ Angle: -90.000°

Available DAT files:
  [0] 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix.dat - X(U)-It_to_PC(100/100)
      Mode: CITS, Type: It_to_PC
      Grid: 100×100
  [1] 20250521_Janus Stacking SiO2_13K_113Lia1R_Matrix.dat - X(U)-Lia1R(100/100)
      Mode: CITS, Type: Lia1R
      Grid: 100×100
  [2] 20250521_Janus Stacking SiO2_13K_113Lia1Y_Matrix.dat - X(U)-Lia1Y(100/100)
      Mode: CITS, Type: Lia1Y
      Grid: 100×100
  [3] 20250521_Janus Stacking SiO2_13K_113Lia2R_Matrix.dat - X(U)-Lia2R(100/100)
      Mode: CITS, Type: Lia2R
      Grid: 100×100


In [5]:
# Create file selector widget
dat_files = list(test_file_path.parent.glob("*.dat"))
dat_files.sort()

file_selector = widgets.Dropdown(
    options=[(f.name, i) for i, f in enumerate(dat_files)],
    value=0,
    description='Select DAT file:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

print("Select a DAT file to analyze:")
display(file_selector)

Select a DAT file to analyze:


Dropdown(description='Select DAT file:', layout=Layout(width='600px'), options=(('20250521_Janus Stacking SiO2…

In [6]:
# Load and parse selected DAT file
selected_file = dat_files[file_selector.value]
print(f"Loading: {selected_file.name}")

# Find corresponding dat_info from txt_data
dat_info = None
for df in dat_files_info:
    if df['filename'] == selected_file.name:
        dat_info = df
        break

if dat_info:
    # Add experiment parameters to dat_info
    dat_info.update({
        'angle': float(exp_info.get('Angle', 0)),
        'x_center': float(exp_info.get('xCenter', 0)),
        'y_center': float(exp_info.get('yCenter', 0))
    })
    
    print(f"  Caption: {dat_info.get('caption', 'N/A')}")
    print(f"  Mode: {dat_info.get('measurement_mode', 'N/A')}")
    print(f"  Type: {dat_info.get('measurement_type', 'N/A')}")
    if dat_info.get('grid_size'):
        print(f"  Grid size: {dat_info['grid_size']}")

# Parse DAT file
print("\nParsing DAT file...")
dat_parser = DatParser()
dat_data = dat_parser.parse(str(selected_file), dat_info)

# Verify CITS data
if is_cits_data(dat_data):
    cits_data = dat_data['data_3d']  # Shape: (n_bias, y, x)
    bias_voltages = dat_data.get('bias_values', None)
    scan_direction = dat_data.get('scan_direction', 'downward')
    
    print(f"\n✓ CITS data confirmed!")
    print(f"  Data shape: {cits_data.shape} (bias={cits_data.shape[0]}, y={cits_data.shape[1]}, x={cits_data.shape[2]})")
    print(f"  Scan direction: {scan_direction}")
    
    # Handle bias voltages
    if bias_voltages is None:
        num_bias = cits_data.shape[0]
        bias_voltages = np.linspace(-1.0, 1.0, num_bias)
        print(f"  Using default bias range: {bias_voltages[0]:.3f}V to {bias_voltages[-1]:.3f}V")
    else:
        # Convert from mV to V if needed
        if dat_data.get('units', {}).get('bias') == 'mV':
            bias_voltages = bias_voltages / 1000.0
        print(f"  Bias range: {bias_voltages[0]:.3f}V to {bias_voltages[-1]:.3f}V")
        print(f"  Number of bias points: {len(bias_voltages)}")
    
    # Prepare data for display
    display_data = prepare_cits_for_display(cits_data, scan_direction)
    print(f"\n✓ Data prepared for display with correct orientation")
    print(f"  Memory efficient: Creates view, not copy")
    print(f"  Shares memory: {np.shares_memory(cits_data, display_data)}")
    
else:
    print("✗ Not CITS data. Please select a different file.")
    cits_data = None
    display_data = None

Loading: 20250521_Janus Stacking SiO2_13K_113It_to_PC_Matrix.dat
  Caption: X(U)-It_to_PC(100/100)
  Mode: CITS
  Type: It_to_PC
  Grid size: [100, 100]

Parsing DAT file...

✓ CITS data confirmed!
  Data shape: (401, 100, 100) (bias=401, y=100, x=100)
  Scan direction: downward
  Bias range: -2.050V to -2.050V
  Number of bias points: 401

✓ Data prepared for display with correct orientation
  Memory efficient: Creates view, not copy
  Shares memory: True


/Users/yangziliang/Git-Projects/keen/backend/core/parsers/dat_parser.py:71: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t', header=None)


## Step 2: Visualize CITS Data at Multiple Bias Voltages

In [7]:
if display_data is not None:
    # Select representative bias indices for visualization
    num_bias = display_data.shape[0]
    indices = [0, num_bias//5, num_bias//3, num_bias//2, 2*num_bias//3, num_bias-1]
    
    # Create subplot figure
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=[f'Bias: {bias_voltages[idx]:.3f}V (#{idx})' for idx in indices],
        horizontal_spacing=0.08,
        vertical_spacing=0.12
    )
    
    # Add heatmaps for each selected bias
    for i, idx in enumerate(indices):
        row = i // 3 + 1
        col = i % 3 + 1
        
        z_data = display_data[idx, :, :]
        
        # Create heatmap
        heatmap = go.Heatmap(
            z=z_data,
            colorscale='RdBu_r',
            showscale=(i == len(indices)-1),  # Show colorbar only for last subplot
            colorbar=dict(
                title=dict(text="Current (A)", side="right"),
                x=1.02,
                len=0.8
            ) if i == len(indices)-1 else None,
            hovertemplate=(
                'X: %{x}<br>' +
                'Y: %{y}<br>' +
                'Current: %{z:.2e} A<br>' +
                '<extra></extra>'
            )
        )
        
        fig.add_trace(heatmap, row=row, col=col)
        
        # Update axes for each subplot
        fig.update_xaxes(title_text="X (pixels)", row=row, col=col)
        fig.update_yaxes(title_text="Y (pixels)", row=row, col=col)
    
    # Update overall layout
    fig.update_layout(
        title=dict(
            text=f"CITS Data Overview - {selected_file.name}<br><sub>Scan: {scan_direction} | Origin at bottom-left | Data corrected for display</sub>",
            x=0.5
        ),
        width=1200,
        height=800,
        showlegend=False
    )
    
    fig.show()
    
    # Print statistics
    print("\nStatistics for displayed bias voltages:")
    print("-" * 70)
    for idx in indices:
        data_slice = display_data[idx, :, :]
        print(f"Bias {bias_voltages[idx]:8.3f}V (#{idx:3d}): "
              f"min={np.min(data_slice):10.3e}, "
              f"max={np.max(data_slice):10.3e}, "
              f"mean={np.mean(data_slice):10.3e}")
    print("-" * 70)
else:
    print("No CITS data available for visualization.")


Statistics for displayed bias voltages:
----------------------------------------------------------------------
Bias   -2.050V (#  0): min=-8.795e-09, max=-1.871e-11, mean=-8.348e-09
Bias   -0.810V (# 80): min=-8.807e-09, max= 1.237e-12, mean=-3.320e-10
Bias    0.011V (#133): min=-8.856e-12, max= 9.787e-10, mean=-1.404e-12
Bias    1.050V (#200): min= 1.848e-10, max= 8.814e-09, mean= 8.476e-09
Bias    0.011V (#267): min=-8.147e-12, max= 5.572e-10, mean=-1.970e-12
Bias   -2.050V (#400): min=-8.840e-09, max=-6.935e-13, mean=-8.386e-09
----------------------------------------------------------------------


## Step 3: Interactive Bias Selection

In [ ]:
if display_data is not None:
    # Create interactive bias selector
    def create_bias_plot(bias_idx):
        """Create a single bias plot"""
        z_data = display_data[bias_idx, :, :]
        
        fig = go.Figure()
        
        fig.add_trace(go.Heatmap(
            z=z_data,
            colorscale='RdBu_r',
            colorbar=dict(
                title=dict(text="Current (A)", side="right")
            ),
            hovertemplate=(
                'X: %{x}<br>' +
                'Y: %{y}<br>' +
                'Current: %{z:.2e} A<br>' +
                '<extra></extra>'
            )
        ))
        
        fig.update_layout(
            title=dict(
                text=f'CITS at Bias: {bias_voltages[bias_idx]:.3f}V (Index: {bias_idx})<br>' +
                     f'<sub>Scan: {scan_direction} | Min: {np.min(z_data):.2e} A | Max: {np.max(z_data):.2e} A</sub>',
                x=0.5
            ),
            xaxis_title="X (pixels)",
            yaxis_title="Y (pixels)",
            width=700,
            height=600,
            template="plotly_white"
        )
        
        # Ensure equal aspect ratio
        fig.update_xaxes(scaleanchor="y", scaleratio=1)
        
        return fig
    
    # Create interactive widget
    bias_slider = widgets.IntSlider(
        value=num_bias//2,  # Start at middle bias
        min=0,
        max=num_bias-1,
        step=1,
        description='Bias Index:',
        continuous_update=False,
        layout={'width': '600px'},
        style={'description_width': 'initial'}
    )
    
    # Output widget for the plot
    output = widgets.Output()
    
    def update_plot(change):
        """Update plot when slider changes"""
        with output:
            clear_output(wait=True)
            fig = create_bias_plot(change['new'])
            fig.show()
    
    # Connect slider to update function
    bias_slider.observe(update_plot, names='value')
    
    # Display initial information
    print("Interactive CITS Bias Selection")
    print("=" * 50)
    print(f"Total bias points: {num_bias}")
    print(f"Bias range: {bias_voltages[0]:.3f}V to {bias_voltages[-1]:.3f}V")
    print(f"Scan direction: {scan_direction}")
    print(f"Grid size: {display_data.shape[1]}×{display_data.shape[2]}")
    print("=" * 50)
    print("\nUse the slider below to explore different bias voltages:")
    
    # Display widgets
    display(bias_slider)
    display(output)
    
    # Show initial plot
    with output:
        fig = create_bias_plot(bias_slider.value)
        fig.show()
        
else:
    print("No CITS data available for interactive visualization.")

Interactive CITS Bias Selection
Total bias points: 401
Bias range: -2.050V to -2.050V
Scan direction: downward
Grid size: 100×100

Use the slider below to explore different bias voltages:


IntSlider(value=200, continuous_update=False, description='Bias Index:', layout=Layout(width='600px'), max=400…

Output()

## Step 4: Performance Analysis - Different Flipping Methods

In [9]:
if cits_data is not None:
    print("=" * 60)
    print("PERFORMANCE ANALYSIS: DATA FLIPPING METHODS")
    print("=" * 60)
    
    # Get data dimensions
    n_bias, grid_y, grid_x = cits_data.shape
    total_points = n_bias * grid_y * grid_x
    
    print(f"Data dimensions: {n_bias} bias × {grid_y} × {grid_x} pixels")
    print(f"Total data points: {total_points:,}")
    print(f"Memory size: ~{cits_data.nbytes / 1024 / 1024:.1f} MB")
    
    # Performance test parameters
    n_iterations = 100
    
    # Method 1: 3D flip (our current approach)
    print("\n1. Testing 3D flip (current approach)...")
    start_time = time.time()
    for _ in range(n_iterations):
        flipped_3d = cits_data[:, ::-1, :]  # Creates a view
    end_time = time.time()
    time_3d_view = (end_time - start_time) / n_iterations * 1000
    
    # Method 2: 3D flip with copy
    print("2. Testing 3D flip with copy...")
    start_time = time.time()
    for _ in range(n_iterations):
        flipped_3d_copy = cits_data[:, ::-1, :].copy()  # Creates a copy
    end_time = time.time()
    time_3d_copy = (end_time - start_time) / n_iterations * 1000
    
    # Method 3: Individual slice flipping
    print("3. Testing individual slice flipping...")
    start_time = time.time()
    for _ in range(n_iterations):
        flipped_slices = np.zeros_like(cits_data)
        for i in range(n_bias):
            flipped_slices[i] = np.flipud(cits_data[i])
    end_time = time.time()
    time_slices = (end_time - start_time) / n_iterations * 1000
    
    # Method 4: 2D reshape + flip (dat_parser style)
    print("4. Testing 2D reshape + flip method...")
    measurement_data_2d = cits_data.reshape(n_bias, -1)
    start_time = time.time()
    for _ in range(n_iterations):
        # Simulate dat_parser approach
        temp_data = measurement_data_2d.copy()
        flipped_2d = temp_data.reshape(-1, grid_y, grid_x)[:, ::-1, :].reshape(-1, grid_x * grid_y)
        result_3d = flipped_2d.reshape(n_bias, grid_y, grid_x)
    end_time = time.time()
    time_2d_reshape = (end_time - start_time) / n_iterations * 1000
    
    # Memory analysis
    print("\n5. Memory behavior analysis...")
    view_result = cits_data[:, ::-1, :]
    copy_result = cits_data[:, ::-1, :].copy()
    
    print(f"  Original data ID: {id(cits_data)}")
    print(f"  View result ID: {id(view_result)} (different object)")
    print(f"  Copy result ID: {id(copy_result)} (different object)")
    print(f"  View shares memory: {np.shares_memory(cits_data, view_result)}")
    print(f"  Copy shares memory: {np.shares_memory(cits_data, copy_result)}")
    
    # Results visualization
    methods = ['3D View', '3D Copy', 'Individual Slices', '2D Reshape']
    times = [time_3d_view, time_3d_copy, time_slices, time_2d_reshape]
    
    # Create performance comparison chart
    fig = go.Figure(data=[
        go.Bar(
            x=methods,
            y=times,
            text=[f'{t:.3f} ms' for t in times],
            textposition='auto',
            marker_color=['green', 'orange', 'red', 'blue']
        )
    ])
    
    fig.update_layout(
        title="Performance Comparison: Data Flipping Methods",
        xaxis_title="Method",
        yaxis_title="Average Time (ms)",
        width=800,
        height=500,
        template="plotly_white"
    )
    
    fig.show()
    
    # Summary
    print("\n" + "=" * 60)
    print("PERFORMANCE SUMMARY:")
    print("=" * 60)
    for method, time_ms in zip(methods, times):
        print(f"{method:18s}: {time_ms:7.3f} ms")
    
    best_idx = np.argmin(times)
    print(f"\n🏆 Best method: {methods[best_idx]} ({times[best_idx]:.3f} ms)")
    
    if best_idx == 0:
        print("\n✅ Our current approach (3D view) is optimal!")
        print("   • Memory efficient (creates view, not copy)")
        print("   • Fastest execution time")
        print("   • Clean and simple code")
    
    print("\n" + "=" * 60)
    print("RECOMMENDATION: Use 3D view approach in prepare_cits_for_display()")
    print("=" * 60)
else:
    print("No CITS data available for performance analysis.")

PERFORMANCE ANALYSIS: DATA FLIPPING METHODS
Data dimensions: 401 bias × 100 × 100 pixels
Total data points: 4,010,000
Memory size: ~30.6 MB

1. Testing 3D flip (current approach)...
2. Testing 3D flip with copy...
3. Testing individual slice flipping...
4. Testing 2D reshape + flip method...

5. Memory behavior analysis...
  Original data ID: 5758510608
  View result ID: 4948787088 (different object)
  Copy result ID: 5489246000 (different object)
  View shares memory: True
  Copy shares memory: False



PERFORMANCE SUMMARY:
3D View           :   0.000 ms
3D Copy           :   5.693 ms
Individual Slices :   9.966 ms
2D Reshape        :   8.760 ms

🏆 Best method: 3D View (0.000 ms)

✅ Our current approach (3D view) is optimal!
   • Memory efficient (creates view, not copy)
   • Fastest execution time
   • Clean and simple code

RECOMMENDATION: Use 3D view approach in prepare_cits_for_display()


## Step 5: STS Line Analysis

In [10]:
if display_data is not None:
    def extract_line_sts(data: np.ndarray, x0: int, y0: int, x1: int, y1: int) -> np.ndarray:
        """
        Extract STS curves along a line from (x0,y0) to (x1,y1)
        
        Args:
            data: 3D CITS data (n_bias, y, x)
            x0, y0: Start coordinates
            x1, y1: End coordinates
            
        Returns:
            np.ndarray: STS curves along the line (n_points, n_bias)
        """
        # Calculate line length and points
        length = max(int(np.sqrt((x1-x0)**2 + (y1-y0)**2)), 1)
        
        if length == 1:
            return np.array([data[:, y0, x0]]).T
        
        # Generate line coordinates
        x_coords = np.linspace(x0, x1, length).astype(int)
        y_coords = np.linspace(y0, y1, length).astype(int)
        
        # Ensure coordinates are within bounds
        x_coords = np.clip(x_coords, 0, data.shape[2]-1)
        y_coords = np.clip(y_coords, 0, data.shape[1]-1)
        
        # Extract STS data along the line
        sts_curves = []
        for x, y in zip(x_coords, y_coords):
            sts_curves.append(data[:, y, x])
        
        return np.array(sts_curves).T  # Shape: (n_bias, n_points)
    
    # Define example line (diagonal across the image)
    margin = 10
    x0, y0 = margin, margin
    x1, y1 = display_data.shape[2] - margin, display_data.shape[1] - margin
    
    print(f"Extracting STS line from ({x0}, {y0}) to ({x1}, {y1})")
    
    # Extract STS curves along the line
    line_sts = extract_line_sts(display_data, x0, y0, x1, y1)
    n_bias_points, n_line_points = line_sts.shape
    
    print(f"Extracted {n_line_points} STS curves with {n_bias_points} bias points each")
    
    # Calculate physical parameters
    x_range = float(exp_info.get('XScanRange', 10.0))
    y_range = float(exp_info.get('YScanRange', 10.0))
    x_pixels = display_data.shape[2]
    y_pixels = display_data.shape[1]
    
    nm_per_pixel_x = x_range / x_pixels
    nm_per_pixel_y = y_range / y_pixels
    line_length_nm = np.sqrt((x1-x0)**2 * nm_per_pixel_x**2 + (y1-y0)**2 * nm_per_pixel_y**2)
    
    print(f"Physical line length: {line_length_nm:.2f} nm")
    
    # Create comprehensive visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Line Selection on CITS Image',
            'Selected STS Curves',
            'STS Evolution Along Line',
            'Line Profile Statistics'
        ),
        specs=[
            [{"type": "heatmap"}, {"type": "scatter"}],
            [{"type": "heatmap"}, {"type": "scatter"}]
        ],
        column_widths=[0.5, 0.5],
        row_heights=[0.5, 0.5],
        horizontal_spacing=0.1,
        vertical_spacing=0.1
    )
    
    # 1. Show line on CITS image (middle bias)
    mid_bias_idx = num_bias // 2
    z_data = display_data[mid_bias_idx, :, :]
    
    fig.add_trace(
        go.Heatmap(
            z=z_data,
            colorscale='RdBu_r',
            showscale=False,
            hovertemplate='X: %{x}<br>Y: %{y}<br>Current: %{z:.2e} A<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Add line overlay
    fig.add_trace(
        go.Scatter(
            x=[x0, x1],
            y=[y0, y1],
            mode='lines+markers',
            line=dict(color='yellow', width=4),
            marker=dict(size=12, color=['lime', 'red']),
            name='Analysis Line',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Plot selected STS curves
    curve_indices = np.linspace(0, n_line_points-1, min(8, n_line_points), dtype=int)
    colors = px.colors.qualitative.Set1[:len(curve_indices)]
    
    for i, idx in enumerate(curve_indices):
        position_nm = (idx / (n_line_points-1)) * line_length_nm if n_line_points > 1 else 0
        
        fig.add_trace(
            go.Scatter(
                x=bias_voltages,
                y=line_sts[:, idx],
                mode='lines',
                name=f'{position_nm:.1f} nm',
                line=dict(color=colors[i], width=2),
                hovertemplate='Bias: %{x:.3f} V<br>Current: %{y:.2e} A<extra></extra>',
                showlegend=True
            ),
            row=1, col=2
        )
    
    # 3. STS Evolution heatmap
    # Subsample for visualization if too many points
    if n_line_points > 100:
        step = n_line_points // 100
        evolution_data = line_sts[:, ::step]
        position_labels = np.linspace(0, line_length_nm, evolution_data.shape[1])
    else:
        evolution_data = line_sts
        position_labels = np.linspace(0, line_length_nm, n_line_points)
    
    fig.add_trace(
        go.Heatmap(
            z=evolution_data,
            x=position_labels,
            y=bias_voltages,
            colorscale='RdBu_r',
            colorbar=dict(
                title="Current (A)",
                x=1.02,
                len=0.4,
                y=0.25
            ),
            hovertemplate='Position: %{x:.1f} nm<br>Bias: %{y:.3f} V<br>Current: %{z:.2e} A<extra></extra>'
        ),
        row=2, col=1
    )
    
    # 4. Line profile statistics
    mean_current = np.mean(line_sts, axis=0)
    std_current = np.std(line_sts, axis=0)
    max_current = np.max(line_sts, axis=0)
    min_current = np.min(line_sts, axis=0)
    
    position_nm_array = np.linspace(0, line_length_nm, n_line_points)
    
    # Mean with error bars
    fig.add_trace(
        go.Scatter(
            x=position_nm_array,
            y=mean_current,
            error_y=dict(type='data', array=std_current, visible=True),
            mode='lines+markers',
            name='Mean ± Std',
            line=dict(color='blue', width=3),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Min/Max envelope
    fig.add_trace(
        go.Scatter(
            x=position_nm_array,
            y=max_current,
            mode='lines',
            name='Max',
            line=dict(color='red', width=1, dash='dash'),
            showlegend=False
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=position_nm_array,
            y=min_current,
            mode='lines',
            name='Min',
            line=dict(color='green', width=1, dash='dash'),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f"STS Line Analysis<br><sub>Line: ({x0},{y0}) → ({x1},{y1}) | Length: {line_length_nm:.1f} nm | {n_line_points} points</sub>",
            x=0.5
        ),
        width=1400,
        height=900,
        template="plotly_white"
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="X (pixels)", row=1, col=1)
    fig.update_yaxes(title_text="Y (pixels)", row=1, col=1)
    fig.update_xaxes(title_text="Bias Voltage (V)", row=1, col=2)
    fig.update_yaxes(title_text="Current (A)", row=1, col=2)
    fig.update_xaxes(title_text="Position (nm)", row=2, col=1)
    fig.update_yaxes(title_text="Bias Voltage (V)", row=2, col=1)
    fig.update_xaxes(title_text="Position (nm)", row=2, col=2)
    fig.update_yaxes(title_text="Current (A)", row=2, col=2)
    
    fig.show()
    
    print(f"\n✓ STS line analysis completed")
    print(f"  Line coordinates: ({x0}, {y0}) → ({x1}, {y1}) pixels")
    print(f"  Physical length: {line_length_nm:.2f} nm")
    print(f"  Number of STS curves: {n_line_points}")
    print(f"  Bias range: {bias_voltages[0]:.3f}V to {bias_voltages[-1]:.3f}V")
    print(f"  Current range: {np.min(line_sts):.2e}A to {np.max(line_sts):.2e}A")
else:
    print("No CITS data available for STS line analysis.")

Extracting STS line from (10, 10) to (90, 90)
Extracted 113 STS curves with 401 bias points each
Physical line length: 11.31 nm



✓ STS line analysis completed
  Line coordinates: (10, 10) → (90, 90) pixels
  Physical length: 11.31 nm
  Number of STS curves: 113
  Bias range: -2.050V to -2.050V
  Current range: -8.83e-09A to 8.81e-09A


## Summary and Conclusions

This notebook demonstrates a complete CITS data analysis workflow with the following key achievements:

### ✅ **Data Orientation Solution**
- Implemented `prepare_cits_for_display()` function for consistent data orientation
- Preserves original data in `dat_parser.py` (no flipping at source)
- Handles orientation correction at analysis/display layer
- Memory efficient (creates view, not copy)

### ✅ **Performance Optimization**
- 3D numpy slicing `[:, ::-1, :]` is the fastest method
- Creates view instead of copy (memory efficient)
- Significantly faster than individual slice processing

### ✅ **Visualization Consistency**
- All plots use Plotly for frontend consistency
- Interactive bias selection with real-time updates
- Proper colorbar configuration
- Origin consistently at bottom-left corner

### ✅ **Analysis Capabilities**
- Multi-bias voltage overview
- Interactive single bias exploration
- STS line analysis with evolution plots
- Statistical analysis along measurement lines

### 🎯 **Next Steps for Integration**
1. Remove redundant `np.flipud()` from `api_mvp.py:494`
2. Integrate `prepare_cits_for_display()` into main application
3. Add STS line analysis to frontend interface
4. Test with upward scan data to verify consistency